<div style="text-align: right">Dec 1<sup>th</sup> 2021</div>

# Programming Tutorial, Day 3
Part of the *Better With Scattering* Workshop by www.bam.de

## Load some data

In [2]:
#!wget "https://raw.githubusercontent.com/BAMresearch/jupyter_tutorial_sas/main/data/Ag_N008.dat"